<a href="https://colab.research.google.com/github/fzehrauslu/ArtificaNeuralNetworkExamples/blob/master/FinalExam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
FolderName="/content/drive/My Drive/Colab Notebooks/HandGestures/acquisitions/S1/G1"

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout
from tensorflow.keras.utils import Sequence
import pdb

Prepare dataset for training

In [ ]:
import os

def getPathsForAClass():
  # ClassNumber :number
  
  pathS1="/content/drive/My Drive/Colab Notebooks/HandGestures/acquisitions/S1"
  pathS2="/content/drive/My Drive/Colab Notebooks/HandGestures/acquisitions/S2"
  pathS3="/content/drive/My Drive/Colab Notebooks/HandGestures/acquisitions/S3"
  pathS4="/content/drive/My Drive/Colab Notebooks/HandGestures/acquisitions/S4"


  G1={"train":[],"validation":[],"test":[]}
  G1["train"]={"Image":[],"label":[]}
  G1["validation"]={"Image":[],"label":[]}
  G1["test"]={"Image":[],"label":[]}
  

  for ClassNumber in [1,5]:#range(1,3): # 12
    ClassNo="G"+str(ClassNumber)
    if ClassNumber==5:
      ClassNumber=2

    for i in range(1,26):
      G1["train"]["Image"].append([os.path.join(pathS1,ClassNo,str(i)+"-color.png" )])
      G1["train"]["label"].append(ClassNumber)
      
      G1["train"]["Image"].append([os.path.join(pathS2,ClassNo,str(i)+"-color.png" )])
      G1["train"]["label"].append(ClassNumber)
      
      G1["train"]["Image"].append([os.path.join(pathS3,ClassNo,str(i)+"-color.png" )])
      G1["train"]["label"].append(ClassNumber)

      G1["train"]["Image"].append([os.path.join(pathS4,ClassNo,str(i)+"-color.png" )])
      G1["train"]["label"].append(ClassNumber)

    for i in range(26,28):
      G1["validation"]["Image"].append([os.path.join(pathS1,ClassNo,str(i)+"-color.png" )])
      G1["validation"]["label"].append(ClassNumber)
      
      G1["validation"]["Image"].append([os.path.join(pathS2,ClassNo,str(i)+"-color.png" )])
      G1["validation"]["label"].append(ClassNumber)
      
      G1["validation"]["Image"].append([os.path.join(pathS3,ClassNo,str(i)+"-color.png" )])
      G1["validation"]["label"].append(ClassNumber)

      G1["validation"]["Image"].append([os.path.join(pathS4,ClassNo,str(i)+"-color.png" )])
      G1["validation"]["label"].append(ClassNumber)


     

    for i in range(28,31):
      G1["test"]["Image"].append([os.path.join(pathS1,ClassNo,str(i)+"-color.png" )])
      G1["test"]["label"].append(ClassNumber)
      
      G1["test"]["Image"].append([os.path.join(pathS2,ClassNo,str(i)+"-color.png" )])
      G1["test"]["label"].append(ClassNumber)
      
      G1["test"]["Image"].append([os.path.join(pathS3,ClassNo,str(i)+"-color.png" )])
      G1["test"]["label"].append(ClassNumber)

      G1["test"]["Image"].append([os.path.join(pathS4,ClassNo,str(i)+"-color.png" )])
      G1["test"]["label"].append(ClassNumber)

      

  return G1

partition=getPathsForAClass()



Write a Dataloader function (40 points)

In [ ]:
## generate dataloader 
## https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

import numpy as np
import keras
import imageio

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True,Training=False):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        #self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.Training=Training

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs["Image"]) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        list_IDs_temp = [self.list_IDs["Image"][k] for k in indexes]
        listLabels=[self.list_IDs["label"][k] for k in indexes]
        
        

        # Generate data
        X, y = self.__data_generation(list_IDs_temp,listLabels,self.Training)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs["Image"]))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp,listLabels,Training):
        'Generates data containing batch_size samples' 
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        #print(list_IDs_temp)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):


            im = imageio.imread(ID[0]) # read png file , color image            
            X[i,] = im

            # Store class number
            y[i] = listLabels[i]-1
        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)


 Set parameters for the **dataloader**

In [ ]:
params = {'dim': (480,640), 
          'batch_size': 8,
          'n_classes': 2, 
          'n_channels': 3, 
          'shuffle': True}

Generate training, validation and test sets

In [ ]:
training_generator = DataGenerator(partition['train'], **params,Training=True)
validation_generator = DataGenerator(partition['validation'], **params,Training=False)
test_generator = DataGenerator(partition['test'], **params,Training=False)

Use Xception pretrained with Imagenet dataset as the base network and add Global Average Pooling, dropout layer, a dense layer with 100 neurons, ReLU layer, a dropout layer and another dense layer with 2 neurons (because we have 2 classes)
and a softmax layer.

In [ ]:
i = tf.keras.layers.Input([None, None, 3], dtype = tf.uint8)
x = tf.cast(i, tf.float32)
x = tf.keras.applications.mobilenet.preprocess_input(x)

# create the base pre-trained model
base_model = Xception(weights='imagenet', include_top=False)
x = base_model(x)#base_model.output
# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)
# add a dropout layer
x= Dropout(0.2)(x)
# let's add a fully-connected layer
x =Dense(100, activation='relu')(x)
# add a dropout layer
x= Dropout(0.2)(x)
# and a dense layer
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model =Model(inputs=i, outputs=predictions) # Model(inputs=base_model.input, outputs=predictions)





Training 1: Train only newly added layers (10 points)

In [ ]:

# set the base model as not trainable
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(training_generator, epochs=5, validation_data=validation_generator)  

# save original model parameters because we need it later when finetuning it with various optimisation methods
tf.keras.models.save_model(
    model, "/content/drive/My Drive/Colab Notebooks/HandGestures/model", overwrite=True, include_optimizer=True, save_format="tf")

Calculate performance metrics

In [ ]:
from sklearn.metrics import classification_report
prediction = model.predict(test_generator)
y_pred_bool = np.argmax(prediction, axis=1)

testDataset=[]
for i in range(len(test_generator)):
  testDataset.append(test_generator[i][1])

testDataset=np.reshape(np.array(testDataset),(-1,2))
y_true_bool = np.argmax(testDataset, axis=1)
print(classification_report(y_true_bool, y_pred_bool))

Training 2: Finetune newly added layers and the late layers of the base model

In [ ]:
# make the selected late layers of the base model trainable
for layer in model.layers[:127]:
   layer.trainable = False
for layer in model.layers[127:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect



Finetune with Adam algorithm and calculate performance metrics (10 points)

In [ ]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer="Adam", loss='categorical_crossentropy',metrics=['accuracy']) 
model.fit(training_generator, validation_data=validation_generator,epochs=5)
prediction=model.predict(test_generator)
y_pred_bool = np.argmax(prediction, axis=1)
print(classification_report(y_true_bool, y_pred_bool))

Finetune with SGD algorithm with alearning rate of 0.0001 and a momentum of 0.7 and calculate performance metrics (10 points)

In [ ]:
loaded_model_1 = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/HandGestures/model')
# make the selected late layers of the base model trainable
for layer in loaded_model_1.layers[:127]:
   layer.trainable = False
for layer in loaded_model_1.layers[127:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
from tensorflow.keras.optimizers import SGD
loaded_model_1.compile(optimizer="Adam", loss='categorical_crossentropy',metrics=['accuracy']) #
loaded_model_1.fit(training_generator, validation_data=validation_generator,epochs=5)
prediction=loaded_model_1.predict(test_generator)
y_pred_bool = np.argmax(prediction, axis=1)
print(classification_report(y_true_bool, y_pred_bool))


Finetune with SGD algorithm with alearning rate of 0.0001 and a momentum of 0.9 and calculate performance metrics (10 points)

In [ ]:
loaded_model_2 = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/HandGestures/model')
# make the selected late layers of the base model trainable
for layer in loaded_model_2.layers[:127]:
   layer.trainable = False
for layer in loaded_model_2.layers[127:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
from tensorflow.keras.optimizers import SGD
loaded_model_2.compile(optimizer="Adam", loss='categorical_crossentropy',metrics=['accuracy']) #
loaded_model_2.fit(training_generator, validation_data=validation_generator,epochs=5)
prediction=loaded_model_2.predict(test_generator)
y_pred_bool = np.argmax(prediction, axis=1)
print(classification_report(y_true_bool, y_pred_bool))

Finetune with SGD algorithm with alearning rate of 0.0001 and a momentum of 0.98 and calculate performance metrics (10 points)

In [ ]:
loaded_model_3 = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/HandGestures/model')
# make the selected late layers of the base model trainable
for layer in loaded_model_3.layers[:127]:
   layer.trainable = False
for layer in loaded_model_3.layers[127:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
from tensorflow.keras.optimizers import SGD
loaded_model_3.compile(optimizer="Adam", loss='categorical_crossentropy',metrics=['accuracy']) 
loaded_model_3.fit(training_generator, validation_data=validation_generator,epochs=5)
prediction=loaded_model_3.predict(test_generator)
y_pred_bool = np.argmax(prediction, axis=1)
print(classification_report(y_true_bool, y_pred_bool))

Compare performance of the finetuned models (10 points)



The best performance was obtained when the model was finetuned with **Adam algorithm**
